<a href="https://colab.research.google.com/github/MahdiTheGreat/Game-playing-systems/blob/main/AI_TTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install -Uqq ipdb
import ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2 MB/s eta 0:00:00
Automatic pdb calling has been turned ON


In [22]:
%pdb off

Automatic pdb calling has been turned OFF


In [19]:

 # MCTS self-play

import numpy as np
import random
import time
from math import sqrt, log
from copy import deepcopy

class MCTSNode:
    def __init__(self, state, parent=None):
        self.state = state  # Tic-Tac-Toe board state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0
        self.untried_moves = self.get_legal_moves()

    def get_legal_moves(self):
        return [(i, j) for i in range(3) for j in range(3) if self.state[i, j] == ""]

    def best_child(self, exploration_weight=1.41):
        return max(self.children, key=lambda child: child.value / (child.visits + 1e-6) + exploration_weight * sqrt(log(self.visits + 1) / (child.visits + 1e-6)))

    def expand(self):
        if not self.untried_moves:
            return self  # Return self if no moves left to expand
        move = self.untried_moves.pop()
        #new_state = self.state.copy()
        new_state = deepcopy(self.state)
        new_state[move] = "X" if np.count_nonzero(self.state == "X") <= np.count_nonzero(self.state == "O") else "O"
        child_node = MCTSNode(new_state, parent=self)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        self.value += result


class MCTS:
    def __init__(self, time_limit=5.0, max_iterations=1000):
        self.time_limit = time_limit
        self.max_iterations = max_iterations

    def search(self, root):
        start_time = time.time()
        iterations = 0

        if root.untried_moves:
            root = root.expand()  # Ensure root expands at least once

        while time.time() - start_time < self.time_limit and iterations < self.max_iterations:
            node = self.select(root)
            if node.untried_moves:
                node = node.expand()
            result = self.simulate(node.state)
            self.backpropagate(node, result)
            iterations += 1

        #return max(root.children, key=lambda c: c.visits, default=root)  # Choose best move by visits
        return root.best_child()  # Choose best move by value

    def select(self, node):
        while node.children and not node.untried_moves:
            node = node.best_child()
        return node

    def simulate(self, state):
        state=deepcopy(state)
        current_player = "X" if np.count_nonzero(state == "X") <= np.count_nonzero(state == "O") else "O"
        while True:
            legal_moves = [(i, j) for i in range(3) for j in range(3) if state[i, j] == ""]
            if not legal_moves:
                return 0  # Draw
            move = random.choice(legal_moves)
            state[move] = current_player
            if self.check_winner(state, current_player):
                return 1 if current_player == "X" else -1
            current_player = "O" if current_player == "X" else "X"

    def backpropagate(self, node, result):
        while node:
            node.update(result)
            result = -result  # Alternate win/loss perspective
            node = node.parent

    def check_winner(self, state, player):
        for i in range(3):
            if all(state[i, :] == player) or all(state[:, i] == player):
                return True
        if all(state.diagonal() == player) or all(np.fliplr(state).diagonal() == player):
            return True
        return False

In [21]:
# MCTS vs user, doesnt work yet

import numpy as np
import random
import time
from math import sqrt, log
from IPython.display import clear_output
from google.colab import output

class TicTacToe:
    def __init__(self,human_player = "X"):
        self.state = np.full((3, 3), "")
        self.mcts = MCTS()

        if human_player == "X":
            self.human_player = "X"
            self.ai_player2 = "O"
        elif human_player == "O":
            self.human_player = "O"
            self.ai_player2 = "X"
        else:
            self.ai_player = "X"
            self.ai_player2= "O"
            self.human_player = None

        self.current_player = "X"

    def display_board(self):
        print("  0 1 2")
        for i in range(len(self.state)):
            print(f"{i} {self.state[i][0]}|{self.state[i][1]}|{self.state[i][2]}")
            if i < 2:
                print("  -+-+-")
        print("\n")

    def get_human_move(self):
     while True:
      try:
          row = int(input("Enter row (0-2): "))
          col = int(input("Enter column (0-2): "))
          if self.state[row, col] == "":
              self.state[row, col] = self.human_player
              self.current_player = self.ai_player2  # Switch turn
              break
          else:
              print("Invalid move, try again!")
      except (ValueError, IndexError):
          print("Invalid input! Enter numbers between 0 and 2.")

    def get_ai_move(self):
     # AI's turn (MCTS)
     print("AI is thinking...")
     time.sleep(1)
     root = MCTSNode(deepcopy(self.state))
     best_move = self.mcts.search(root)
     #ipdb.set_trace()
     self.state = best_move.state
     self.current_player = self.human_player  # Switch turn

    def play_game(self):
        while True:
            clear_output()
            print("current state: ",self.state)
            self.display_board()

            first_player=self.human_player if self.human_player else self.ai_player
            first_player_move = self.get_human_move() if self.human_player else self.get_ai_move()

            if self.current_player == first_player:
              first_player_move()
            else:
              self.get_ai_move()

            if self.mcts.check_winner(self.state, "X"):
                self.display_board()
                print("X wins!")
                break
            elif self.mcts.check_winner(self.state, "O"):
                self.display_board()
                print("O wins!")
                break
            elif "" not in self.state:
                self.display_board()
                print("It's a draw!")
                break

if __name__ == "__main__":
    game = TicTacToe()
    game.play_game()


current state:  [['X' '' 'X']
 ['' 'X' 'O']
 ['' 'O' 'O']]
  0 1 2
0 X||X
  -+-+-
1 |X|O
  -+-+-
2 |O|O




KeyboardInterrupt: Interrupted by user

> /usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py(1219)_input_request()
   1217             except KeyboardInterrupt:
   1218                 # re-raise KeyboardInterrupt, to truncate traceback
-> 1219                 raise KeyboardInterrupt("Interrupted by user") from None
   1220             except Exception:
   1221                 self.log.warning("Invalid Message:", exc_info=True)

ipdb> exit
